# HW 4: Replication of Rust (1987)

<h3> Hasan K. Tosun, December 2017 </h3>

In this homework, we are asked to estimate the model described in Rust (1987). However, by using a different method than the one in the original paper. 

I'll use the method described in the paper first. Then, I'll move on to the "stata" estimator. (I don't know why they name it this way.)

Before going into detail, let me sketch what I'm going to do.

I'll use the <a href="http://www.github.com/hktosun/hktosun.github.io/blob/master/coursework/IO/ECON8602/stata_rust_data.dta">data</a> provided by Prof. Amil Petrin. 

In the first section, I'll do the following:
1. Pick a functional form for the cost function.
2. Estimate $p(x'\mid x,d)$ by using the data.
3. Find the fixed point of the contraction for $EV_\theta$.
4. Find $P(d\mid x,\theta)$ by using $EV_\theta$.
5. Calculate the likelihood function.
6. Find parameter values $\theta$ that maximizes log-likelihood function.

In the second section, I'll follow these steps:
1. Estimate $p(x'\mid x,d)$ by using the data.
2. Estimate $p(d\mid x)$.
3. Iterate on the fixed point relation to get $v(x,1)$.
4. Use $v(x,1)$ to get $v(x,d)$.
5. Find $u(x,d)$ by using $v(x,d)$.

Let's get started.

## Part 1: Estimation of Parameters by Maximizing Partial Loglikelihood

Step 0: Import the necessary packages, import the data, set some parameters.

In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from numba import jit
import time

data = pd.read_csv("rust.csv")

cts = data['frag'].value_counts(sort = False)
nrow = data.shape[0]
pi = cts/nrow

beta = 0.9999

n = 90
x = np.array(range(1,n+1))
x.shape = (n,1)

tol = 1e-6
 

Step 1: Define a function that will calculate the cost of operating in state $x$, given the structural parameter $\theta_1$. I pick the linear formulation as the functional form for the cost.

In [3]:
@jit
def calc_cost(theta1):
    c = theta1 * x
    return(c)

Step 2: Define a function that will construct the probability transition matrix $P$. This is an $n\times n$ matrix, governing the motion of $x$'s. 

In [4]:
@jit
def probs():
    P = np.zeros((n,n))
    for i in range(n-2):
        P[i,i] = pi[0]
        P[i,i+1] = pi[1]
        P[i,i+2] = pi[2]
    P[n-2,n-2] = pi[0]
    P[n-2,n-1] = 1 - pi[0]
    P[n-1,n-1] = 1
    return(P)

Step 3: Define the contraction function, given the structural parameters $\theta_1$ and $tr$, total cost for replacement.

In [5]:
@jit
def inside_contract(theta1,tr,ev):
    c = calc_cost(theta1)
    temp = np.exp(-c + beta * ev - ev) + np.exp(-tr - c[0,0] + beta * ev[0,0] - ev)
    return (np.log(temp) + ev)

@jit
def contraction(theta1, tr):
    dif = 1
    ev = np.zeros((n,1))
    while (dif > tol):
        ev1 = P @ inside_contract(theta1,tr,ev)
        dif = (abs(ev1-ev)).max()
        ev = ev1
    return(ev)

Step 4: Calculate $P(0\mid x; \theta_1,tr)$.

In [6]:
@jit
def P0x(theta1, tr):
    c = calc_cost(theta1)
    ev = contraction(theta1, tr)
    pk = 1/(1 + np.exp(c - beta * ev - tr - c[0,0] + beta * ev[0,0]))
    return(pk)

Step 5: Calculate the partial loglikelihood function:
$$
\begin{equation}
LL(d,x;\theta_1,tr) = \sum_{m=1}^M \sum_{t=1}^T \log P(d_{mt}\mid x_{mt};\theta_1,tr)
\end{equation}
        $$

In [7]:
@jit
def part_likelihood(params):
    theta = params[0]
    tr = params[1]
    #print("{}  {}".format(theta,tr))
    temp = 0
    p0x = P0x(theta, tr)
    for j in range(8260):
        x_ind = data.bxt[j] - 1
        i = data.i[j]
        temp = temp + np.log(p0x[x_ind]*(i==0) + (1-p0x[x_ind])*(i==1)+0.0000001)
    temp = temp[0,]
    #print("{}".format(temp))
    return(-temp)

Step 6: Maximization of Partial Loglikelihood Function

$$
\begin{equation}
\max_{\theta_1,tr} LL(d,x;\theta_1,tr)
\end{equation}
$$

In [8]:
P = probs()
init = np.array([0.001,10])

t = time.time()
bnds = ((0.0001, 0.1), (1, 40))
soln = minimize(part_likelihood, init, method="L-BFGS-B", bounds = bnds, options={'disp': True})
elapsed_time = time.time()-t

In [9]:
print("Time elapsed:\t",elapsed_time)
print("Loglikelihood:\t", -soln.fun)
print("theta1:\t\t", soln.x[0])
print("TR:\t\t",soln.x[1])

Time elapsed:	 235.20060396194458
Loglikelihood:	 -299.18982815
theta1:		 0.0026573741074
TR:		 9.80632865721


## Part 2: Estimation of the Utility Function

Suppose, instead, that we want to estimate the utility function altogether, rather than estimating the replacement cost and the operating cost parameters assuming a functional form for the operating cost. 

We have the same fixed point condition as before:

$$
\begin{equation}
v(x,d) = u(x,d) + \beta \int \log \left(\sum_{d'\in D} \exp(v(x',d'))\right) p(x'|x,d)dx'
\end{equation}
$$

Add and subtract $\beta \int v(x'|1) p(x'|x,d) dx'$ on the RHS:

$$
\begin{align}
v(x,d) & = u(x,d) + \beta \int \log \left(\sum_{d'\in D} \exp(v(x',d'))\right) p(x'|x,d)dx' - \beta \int v(x'|1) p(x'|x,d) dx' + \beta \int v(x'|1) p(x'|x,d) dx' \\
& = u(x,d) + \beta \int \left(\log \left(\sum_{d'\in D} \exp(v(x',d'))\right) - \log(\exp(v(x',1))) \right) p(x'|x,d)dx'  + \beta \int v(x'|1) p(x'|x,d) dx' \\
& = u(x,d) + \beta \int \log \left( \frac{\sum_{d'\in D} \exp(v(x',d'))}{\exp(v(x',1))}\right) p(x'|x,d)dx'  + \beta \int v(x'|1) p(x'|x,d) dx' \\
& = u(x,d) + \beta \int \log \left( \sum_{d'\in D} \exp(v(x',d')-v(x',1))\right) p(x'|x,d)dx'  + \beta \int v(x'|1) p(x'|x,d) dx' \\
\end{align}
$$

We can do further manipulations on this equation. How? Remember that we have a multinomial logit structure. So,

$$
\begin{equation}
p(d|x) = \frac{\exp(v(x,d))}{\sum_{d'\in D} \exp(v(x,d'))}
\end{equation}
$$

So, we can write 

$$
\begin{equation}
p(1|x) = \frac{\exp(v(x,1))}{\sum_{d'\in D} \exp(v(x,d'))}
\end{equation}
$$

Then,
$$
\begin{align}
\frac{p(d|x)}{p(1|x)} & = \dfrac{\dfrac{\exp(v(x,d))}{\sum_{d'\in D} \exp(v(x,d'))}}{\dfrac{\exp(v(x,1))}{\sum_{d'\in D} \exp(v(x,d'))}}\\
& = \frac{\exp(v(x,d))}{\exp(v(x,1))}
\end{align}
$$

Taking logs of both sides, we get

$$
\begin{equation}
\log(p(d|x)) - \log(p(1|x)) = v(x,d) - v(x,1)
\end{equation}
$$

Now, replace $v(x',d') - v(x',1)$ in the above fixed point equation with $\log(p(d'|x')) - \log(p(1|x'))$.

$$
\begin{align}
v(x,d) & = u(x,d) + \beta \int \log \left( \sum_{d'\in D} \exp(\log(p(d'|x')) - \log(p(1|x')))\right) p(x'|x,d)dx'  + \beta \int v(x'|1) p(x'|x,d) dx' \\
& = u(x,d) + \beta \int \log \left( \sum_{d'\in D} \frac{p(d'|x')}{p(1|x')}\right) p(x'|x,d)dx'  + \beta \int v(x'|1) p(x'|x,d) dx' \\
& = u(x,d) + \beta \int \log \left( \frac{1}{p(1|x')} \sum_{d'\in D} p(d'|x')\right) p(x'|x,d)dx'  + \beta \int v(x'|1) p(x'|x,d) dx' \\
& = u(x,d) + \beta \int \log \left( \frac{1}{p(1|x')}\right) p(x'|x,d)dx'  + \beta \int v(x'|1) p(x'|x,d) dx' \\
& = u(x,d) - \beta \int \log p(1|x') p(x'|x,d)dx'  + \beta \int v(x'|1) p(x'|x,d) dx' \\
\end{align}
$$

Normalize $u(x,1) = 0$, and write the above equation for $d=1$;
$$
\begin{equation}
v(x,1) = - \beta \int \log p(1|x') p(x'|x,1)dx'  + \beta \int v(x'|1) p(x'|x,1) dx'
\end{equation}
$$

Plan of attack:

1. Estimate $p(x'|x,d)$.
2. Estimate $p(d|x)$.
3. Find the fixed point for $v(x,1)$ using above equation.
4. Use $v(x,d) = \log p(d|x) - \log p(1|x) + v(x|1)$ to estimate $v(x,d)$.
5. Use $u(x,d) = v(x,d) - \beta \int \log \left(\sum_{d'\in D} \exp(v(x',d'))\right) p(x'|x,d)dx'$ to estimate $u(x,d)$.


In [10]:
data.i == 

,model,group,year,month,i,xt,xt1,odometer,xtmxt1,bxt,bxt1,xtchange,frag
0,4403,1,83,5,0,0.0,504.0,504.0,504.0,1,1,0,0
1,4403,1,83,6,0,504.0,2705.0,2705.0,2201.0,1,1,0,0
2,4403,1,83,7,0,2705.0,7345.0,7345.0,4640.0,1,2,1,1
3,4403,1,83,8,0,7345.0,11591.0,11591.0,4246.0,2,3,1,1
4,4403,1,83,9,0,11591.0,16057.0,16057.0,4466.0,3,4,1,1
5,4403,1,83,10,0,16057.0,20796.0,20796.0,4739.0,4,5,1,1
6,4403,1,83,11,0,20796.0,25299.0,25299.0,4503.0,5,6,1,1
7,4403,1,83,12,0,25299.0,29311.0,29311.0,4012.0,6,6,0,0
8,4403,1,84,1,0,29311.0,34621.0,34621.0,5310.0,6,7,1,1
9,4403,1,84,2,0,34621.0,39738.0,39738.0,5117.0,7,8,1,1
